## Deep Learning approach using NLP models

 Import Required Libraries

In [37]:
!pip install tensorflow
!pip install keras


Load the Dataset and preprocess

In [58]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
file_path = '/content/grammar_correction_pairs.csv'
data = pd.read_csv(file_path)

# Preprocessing function to clean the text
def preprocess_text(text):

    text = re.sub(r'[^A-Za-z0-9\s\u0D80-\u0DFF.,?!]', '', text)
    text = text.lower()
    return text

# Apply preprocessing to both incorrect and correct columns
data['incorrect_sentence'] = data['incorrect_sentence'].apply(preprocess_text)
data['correct_sentence'] = data['correct_sentence'].apply(preprocess_text)

# Check the preprocessed data
data.head()


,incorrect_sentence,correct_sentence
0,මම යන්නෙමු,මම යන්නෙමි
1,මම යන්නෙමුවා,මම යන්නෙමි
2,මම යන්නෙහි,මම යන්නෙමි
3,මම යන්නෙහිවා,මම යන්නෙමි
4,මම යන්නේය,මම යන්නෙමි


Modify Tokenization and Padding

In [59]:
# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the incorrect sentences (paragraphs)
tokenizer.fit_on_texts(data['incorrect_sentence'])

# Convert the sentences (paragraphs) to sequences
incorrect_sequences = tokenizer.texts_to_sequences(data['incorrect_sentence'])
correct_sequences = tokenizer.texts_to_sequences(data['correct_sentence'])

# Pad the sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in incorrect_sequences])  # Find the longest sequence

X = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post')
y = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post')

# Check the shape of the sequences
X.shape, y.shape


((14380, 7), (14380, 7))

Create the Model

In [66]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Attention

# Define the model architecture with attention mechanism
input_seq = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_seq)

# Encoder LSTM
encoder_lstm = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(embedding_layer)

# Decoder LSTM with attention mechanism
decoder_lstm = LSTM(128, return_sequences=True)(embedding_layer, initial_state=[state_h, state_c])
attention = Attention()([decoder_lstm, decoder_lstm])

# Output layer for generating words in the sequence
output = TimeDistributed(Dense(len(tokenizer.word_index) + 1, activation='softmax'))(attention)

# Define the model
model = Model(inputs=input_seq, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summarize the model architecture
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 7)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_11 (Embedding)  │ (None, 7, 128)         │         25,088 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_14 (LSTM)            │ [(None, 128), (None,   │        131,584 │ embedding_11[0][0]     │
│                           │ 128), (None, 128)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_15 (LSTM)            │ (None, 7, 128)         │        131,584 │ embedding_11[0][0],    │
│                           │                        │                │ lstm_14[0][1],         │
│                           │                        │                │ lstm_14[0][2]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_2 (Attention)   │ (None, 7, 128)         │              0 │ lstm_15[0][0],         │
│                           │                        │                │ lstm_15[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_9        │ (None, 7, 196)         │         25,284 │ attention_2[0][0]      │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 313,540 (1.20 MB)

 Trainable params: 313,540 (1.20 MB)

 Non-trainable params: 0 (0.00 B)

 Train the Model

In [61]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# One-hot encode the target sequences
y_one_hot = np.array([to_categorical(seq, num_classes=len(tokenizer.word_index) + 1) for seq in y])

# Train the model
model.fit(X, y_one_hot, epochs=10, batch_size=64)


Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.3075 - loss: 3.4090
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.8519 - loss: 0.7032
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.9164 - loss: 0.3718
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 55ms/step - accuracy: 0.9630 - loss: 0.1993
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - accuracy: 0.9750 - loss: 0.1279
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 57ms/step - accuracy: 0.9831 - loss: 0.0858
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9887 - loss: 0.0568
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 56ms/step - accuracy: 0.9917 - loss: 0.0416
Epoch 9/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 18s 47ms/step - accuracy: 0.9943 - loss: 0.0299
Epoch 10/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.9969 - loss: 0.0196


Grammar Error Correction on Test Data

In [63]:
# Example input paragraphs (with grammatical errors)
test_paragraphs = [
    "මම පාසල් යමු.අපි පාසල් යමු.මම ගමට යමි.අපි ගමට යමි.",
    "මම අකුරු ලියමි.අපි අකුරු ලියමි.මම පාඩම් කරමි.අපි පාඩම් කරමු.",
    "මම ගමට යමි.අපි ගමට යමු.මම ගෙදර යනවා.අපි ගෙදර යහි.",
    "මම දුවමි.අපි දුවමු.මම යයි.අපි යති.",
    "මම මල් කඩයි.මම මල් කඩමි.අපි මල් කඩමු.අපි මල් කඩති."
]

# Preprocess the test paragraphs
test_paragraphs = [preprocess_text(paragraph) for paragraph in test_paragraphs]

# Convert the test paragraphs to sequences
test_sequences = tokenizer.texts_to_sequences(test_paragraphs)

# Pad the test sequences
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict the corrections using the trained model
predictions = model.predict(X_test)

# Convert predictions back to words
def decode_sequence(sequence):
    return ' '.join([tokenizer.index_word.get(i, '') for i in sequence])

# Decode and display the corrected paragraphs
for i, paragraph in enumerate(test_paragraphs):
    corrected_paragraph = decode_sequence(np.argmax(predictions[i], axis=-1))
    print(f"Original: {paragraph} -> Corrected: {corrected_paragraph}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Original: මම පාසල් යමු.අපි පාසල් යමු.මම ගමට යමි.අපි ගමට යමි. -> Corrected: ලස්සන අපි යන්නෙමු මම යන්නෙමුවා යැවෙමි යන්නෙමුවා
Original: මම අකුරු ලියමි.අපි අකුරු ලියමි.මම පාඩම් කරමි.අපි පාඩම් කරමු. -> Corrected: වාහන මම ගෙදර ගෙදර   
Original: මම ගමට යමි.අපි ගමට යමු.මම ගෙදර යනවා.අපි ගෙදර යහි. -> Corrected: වාහන මම ගෙදර යැවෙමි යැවෙමි ගෙදර යවමි
Original: මම දුවමි.අපි දුවමු.මම යයි.අපි යති. -> Corrected: මම මම මම යමිවා මම යැවෙමි 
Original: මම මල් කඩයි.මම මල් කඩමි.අපි මල් කඩමු.අපි මල් කඩති. -> Corrected: වාහන වාහන ගෙදර ගෙදර   


Model Evaluation

In [53]:
# Evaluate the model on a test set (if available)
model.evaluate(X_test, y_one_hot_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
Original: මම ගමට යමු. අපි ගමට යමි. ඔහු කුඩා කෙනෙකි. ඇය ගෙදර යයි. -> Corrected: මම මම අපි මම ගෙදර යමිවා 
Original: මම පාසල් යමු.අපි පාසල් යමු.මම ගමට යමි.අපි ගමට යමි -> Corrected: මම මම මම මම යමිවා මම යමිවා
Original: මම අකුරු ලියමි .අපි අකුරු ලියමි.මම පාඩම් කරමි.අපි පාඩම් කරමු -> Corrected: මම මම මම මම   
Original: ඔවුන් ව්යාපාරයක් ආරම්භ කරති. මට එය කරන්න පුළුවන්. ඔහු මට හෝ දෙයක් අමතක කරන්නේ. අපි කෙනෙකුට පිළිතුරු ලැබීමේ ඉක්මනින් -> Corrected: මම      
Original: අපි අලුත්ම වැඩක් කරන්න යනවා. ඔහුට මේ සියලු දේ හොඳින් අවබෝධයි. එය කාර්ය මණ්ඩලයේ අලුත්ම උපායමයි. ඔවුන් අපිට බලන්න ඉඩ දුන්නේ. -> Corrected: මම යැවෙමි ගෙදර    
